In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime
import os

import geopandas as gpd
import keras

import gee
import utils

In [ ]:
region_name = 'tapajos_basin'

tile_size = 576 # this is the around the max size that GEE exports allow with 12-band imagery
tile_padding = 24

start_date = datetime(2022, 1, 1)
end_date = datetime(2022, 12, 31)
clear_threshold = 0.6

pred_threshold = 0.5

In [ ]:
# load an ensembled model
model_name = '48px_v3.7-ensemble_2024-01-17'
model = keras.models.load_model(f'../models/{model_name}.h5')
region = gpd.read_file(f'../data/boundaries/{region_name}.geojson').geometry[0].__geo_interface__
tiles = utils.create_tiles(region, tile_size, tile_padding)
print(f"Created {len(tiles):,} tiles")

In [ ]:
data_pipeline = gee.S2_Data_Extractor(
    tiles, 
    start_date, 
    end_date, 
    clear_threshold, 
    batch_size=500
    )

In [ ]:
data_pipeline.make_predictions(model, pred_threshold=pred_threshold)

In [ ]:
# write the predictions to a file
print(len(data_pipeline.predictions), 'chips with predictions above', pred_threshold)
# write the predictions to a file
model_version_name = '_'.join(model_name.split('_')[0:2])
# if the outputs directory does not exist, create it
if not os.path.exists(f'../data/outputs/{model_version_name}'):
    os.makedirs(f'../data/outputs/{model_version_name}')
time_period = f"{start_date.month}_{start_date.year}-{end_date.month}_{end_date.year}"
data_pipeline.predictions.to_file(f"../data/outputs/{model_version_name}/{region_name}_{model_version_name}_{pred_threshold:.2f}_{time_period}.geojson", driver="GeoJSON")